# COST INTERACT ML Challenge (NET) - Baseline

The code contained in this notebook serves as the baseline code provided by the organizers of the COST INTERACT ML challenge (NET). The baseline implements a shallow neural network with gaussian output as a probabilistic regressor. The baseline is provided as a starting point for participants to build upon.

Authors: Marco Skocaj (HA1 Chair, Università di Bologna, Italy), Nicola Di Cicco (Politecnico di Milano, Italy)

In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf # install tf with pip install tensorflow==2.10
import tensorflow_probability as tfp # install tfp with pip install tensorflow-probability==0.18
from tqdm import tqdm

# Check tensorflow version (tf <= 2.10 required for native gpu support on Windows)
print(tf.__version__)

# Check tf is running on gpu
print(tf.config.list_physical_devices('GPU'))

2.10.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# Your local starting kit path here
data_fpath = r"C:\Users\skoca\PycharmProjects\starting_kit_NET"
fpath_train = f"{data_fpath}/train.csv"
fpath_val = f"{data_fpath}/val_no_labels.csv"
fpath_test = f"{data_fpath}/test_no_labels.csv"

# Read data
train = pd.read_csv(fpath_train)
val = pd.read_csv(fpath_val)
test = pd.read_csv(fpath_test)

In [3]:
# Define a probabilistic model. In this example we use a shallow probabilistic neural network with a gaussian output
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2),
    tfp.layers.DistributionLambda(
        lambda t: tfp.distributions.Normal(loc=t[..., :1],
                                           scale=1e-3 + tf.math.softplus(0.05 * t[..., 1:]))),
])

# Define loss function (negative log likelihood)
def nll(y_true, dist):
    return -dist.log_prob(y_true)

# Build model
model.build(input_shape=(None, 3))

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=nll)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                128       
                                                                 
 dense_1 (Dense)             (None, 2)                 66        
                                                                 
 distribution_lambda (Distri  ((None, 1),              0         
 butionLambda)                (None, 1))                         
                                                                 
Total params: 194
Trainable params: 194
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Train model
print("Training model...")
model.fit(train.drop('UL Throughput (Mbps)', axis=1), train['UL Throughput (Mbps)'], epochs=20, verbose=1)

### Compute metrics ###
# Perform an MC experiment to get the predictive distribution
n_experiments = 50
print("Predicting...")
preds_val = np.stack([model.predict(val, verbose=0) for _ in tqdm(range(n_experiments))]).squeeze()
preds_test = np.stack([model.predict(test, verbose=0) for _ in tqdm(range(n_experiments))]).squeeze()

# Compute a point estimate of your predictive distribution. In this case, the mean of the Gaussian
preds_val_mean = np.mean(preds_val, axis=0)

# Compute the 80% CI of the predictive distribution, low and high bounds
preds_val_low_80 = np.quantile(preds_val, 0.1, axis=0)
preds_val_high_80 = np.quantile(preds_val, 0.9, axis=0)

# Compute the 90% CI of the predictive distribution, low and high bounds
preds_val_low_90 = np.quantile(preds_val, 0.05, axis=0)
preds_val_high_90 = np.quantile(preds_val, 0.95, axis=0)

# Compute the 95% CI of the predictive distribution, low and high bounds
preds_val_low_95 = np.quantile(preds_val, 0.025, axis=0)
preds_val_high_95 = np.quantile(preds_val, 0.975, axis=0)

# Create a dataframe with the predictions metrics
preds_val_df = pd.DataFrame({'point': preds_val_mean,
                             'lo_80': preds_val_low_80,
                             'hi_80': preds_val_high_80,
                             'lo_90': preds_val_low_90,
                             'hi_90': preds_val_high_90,
                             'lo_95': preds_val_low_95,
                             'hi_95': preds_val_high_95})

preds_val_df

Training model...
Epoch 1/20
67/67 [==============================] - 2s 3ms/step - loss: 0.8248
Epoch 2/20
67/67 [==============================] - 0s 3ms/step - loss: 0.9557
Epoch 3/20
67/67 [==============================] - 0s 3ms/step - loss: 0.4673
Epoch 4/20
67/67 [==============================] - 0s 3ms/step - loss: 0.3654
Epoch 5/20
67/67 [==============================] - 0s 3ms/step - loss: 0.1487
Epoch 6/20
67/67 [==============================] - 0s 2ms/step - loss: 0.2649
Epoch 7/20
67/67 [==============================] - 0s 3ms/step - loss: 0.0749
Epoch 8/20
67/67 [==============================] - 0s 2ms/step - loss: -0.1732
Epoch 9/20
67/67 [==============================] - 0s 2ms/step - loss: -0.3910
Epoch 10/20
67/67 [==============================] - 0s 3ms/step - loss: -0.6364
Epoch 11/20
67/67 [==============================] - 0s 2ms/step - loss: -0.8460
Epoch 12/20
67/67 [==============================] - 0s 3ms/step - loss: -0.4568
Epoch 13/20
67/67 [=======

100%|██████████| 50/50 [00:03<00:00, 14.47it/s]


,point,lo_80,hi_80,lo_90,hi_90,lo_95,hi_95
0,0.085019,0.019817,0.140848,-0.018095,0.162632,-0.023904,0.210095
1,0.095311,0.013366,0.186220,0.008445,0.203451,0.003010,0.216515
2,0.084715,-0.086970,0.260772,-0.130551,0.289418,-0.162904,0.323550
3,0.094406,0.018040,0.165642,-0.009483,0.176935,-0.015448,0.188620
4,0.104588,-0.011762,0.197887,-0.033109,0.201371,-0.054417,0.203565
...,...,...,...,...,...,...,...
450,-5.402172,-6.665071,-4.116567,-7.272796,-3.656495,-7.445249,-3.600164
451,-2.667571,-3.497131,-1.926564,-3.779995,-1.536213,-4.022359,-1.403440
452,-1.910905,-2.739428,-1.155323,-3.030740,-1.085644,-3.177084,-1.057540
453,-5.897260,-7.403997,-4.548115,-8.211551,-4.222713,-8.537130,-4.183210


In [5]:
# Save predictions to csv file and zip it.
#  The zipped file should be used for score submission on Codalab.
preds_val_df.to_csv(f"{data_fpath}/preds.csv", index=False)
is_written = os.system(f"zip preds.zip preds.csv")

if ~is_written:
    print("Predictions saved to preds.zip")
else:
    print("Error while saving predictions to preds.zip")

Predictions saved to preds.zip


In [6]:
# You can test the logic of the scoring mechanism on Codalab here.
#  Just be sure to keep a held-out validation test from the original training set with features and labels.

def score(ground_truth: pd.DataFrame, preds: pd.DataFrame):
    # Check that the submission file has the correct headers
    expected_headers = ["point", "lo_80", "hi_80", "lo_90", "hi_90", "lo_95", "hi_95"]
    preds_headers = list(preds.columns)
    if(set(expected_headers) != set(preds_headers)):
        print(f"Submission file headers do not match expected headers. Please double-check the submission file.\nExpected headers: {expected_headers}\nSubmission file headers: {preds_headers}")
        return 404, 404, 404

    ci_levels = [80, 90, 95]
    ul_thp = ground_truth["UL Throughput (Mbps)"]
    calibration_errors = []
    sharpnesses = []
    
    # Compute MAE
    mae = np.mean(np.abs(ul_thp - preds["point"]))
    if(np.isnan(mae)):
        print(f"MAE is NaN, submission file invalid. Please double-check the submission file.")
        return 404, 404, 404

    # Compute coverage and sharpness for each confidence level
    for level in ci_levels:
        hi_level = preds[f"hi_{level}"]
        lo_level = preds[f"lo_{level}"]

        # Compute the coverage
        # Calculate the quantiles for the centered confidence intervals
        offset = (100 - level) / 2
        quantile_lo = offset / 100
        quantile_hi = 1 - offset / 100
        # sanity check
        assert np.isclose(quantile_hi - quantile_lo, level / 100)

        # Calculate the empirical coverage for both quantiles
        coverage_lo = np.mean(ul_thp <= lo_level)
        coverage_hi = np.mean(ul_thp <= hi_level)

        # Calculate the calibration errors
        calibration_error_lo = np.abs(coverage_lo - quantile_lo)
        calibration_error_hi = np.abs(coverage_hi - quantile_hi)
        calibration_errors.append((calibration_error_lo + calibration_error_hi) / 2)

        # Compute the sharpness
        sharpness = np.mean(hi_level - lo_level)
        sharpnesses.append(sharpness)
        if(np.isnan(sharpness)):
            print(f"Sharpness at {level}% is NaN, submission file invalid. Please double-check the submission file.")
            return 404, 404, 404
    
    
    print("MAE: ", mae)
    print("Coverage (80% CI, 90% CI, 95% CI): ", calibration_errors)
    print("Sharpness (80% CI, 90% CI, 95% CI): ", sharpnesses)

truth = pd.read_csv(r"C:\Users\skoca\PycharmProjects\MLcomp_INTERACT_NET_bundle\reference_data_val\val_ref.csv")
pred = pd.read_csv(f"{data_fpath}/preds.csv")

score(truth, pred)

MAE:  0.11347505576042179
Coverage (80% CI, 90% CI, 95% CI):  [0.07142857142857142, 0.03351648351648354, 0.013186813186813197]
Sharpness (80% CI, 90% CI, 95% CI):  [0.42768791651496524, 0.5426242482633543, 0.6266905651593598]
